<a href="https://colab.research.google.com/github/sayanbanerjee32/NLP-with-fastai2.0/blob/main/fastai_on_20newsgroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install fastai --upgrade

In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

In [3]:
def twenty_newsgroup_to_df(data_set = 'train'):
    newsgroups_train = fetch_20newsgroups(subset=data_set, 
                                          remove=('headers', 'footers',
                                                  'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame(newsgroups_train.target_names)
    targets.columns = ['title']

    train = pd.merge(df, targets, left_on='target', right_index=True)
    #out['date'] = pd.to_datetime('now')
    #out.to_csv('20_newsgroup.csv')
    return train


train_data_df = twenty_newsgroup_to_df('train')
test_data_df = twenty_newsgroup_to_df('test')

In [4]:
train_data_df.info(),test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11314 entries, 0 to 11302
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11314 non-null  object
 1   target  11314 non-null  object
 2   title   11314 non-null  object
dtypes: object(3)
memory usage: 353.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7532 entries, 0 to 7519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7532 non-null   object
 1   target  7532 non-null   object
 2   title   7532 non-null   object
dtypes: object(3)
memory usage: 235.4+ KB


(None, None)

In [5]:
from fastai.text.all import *

In [6]:
dls = TextDataLoaders.from_df(train_data_df, text_col='text', label_col='target')
dls.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj due to the resolution and size it is in 14 parts . \n\n xxmaj this is a uuencoded bitmap . xxunk 256 colors . \n xxmaj the picture is a xxunk xxunk on a desert with blue sky background . \n xxmaj the size is just right for centered wallpaper on a 1024x768 display \n because it leaves a border at the bottom just big enough for icons \n to fit in without being on top of the picture . xxmaj xxunk image \n quality and resolution - i have not seen much better . \n\n xxmaj for those of you who have n't worked with xxunk image files , here \n is how to put it back together . \n\t 1 . save the 14 parts to 14 individual files \n\t 2 . use a text editor to remove the header and xxunk \n\t▁ in each file",2
1,"xxbos xxmaj try the firearms archive . xxmaj larry xxmaj cipriani 's instructions follow . xxmaj by \n the way , thanks for the archive xxmaj larry .. \n\n xxmaj this year is the 103rd congress directory . \n\n▁ xxrep 28 - \n xxmaj from \n xxunk \n xxunk . \n xxunk xxmaj thu xxmaj apr 8 xxunk 1993 \n xxmaj article : xxunk of talk.politics.guns \n xxmaj path : \n xxunk \n xxunk . \n xxunk \n xxmaj from : xxunk \n xxmaj newsgroups : talk.politics.guns \n xxmaj subject : xxmaj information about the anonymous ftp xxup rkba archive \n message - id : < xxunk > \n xxmaj date : 8 xxmaj apr 93 xxunk xxup gmt \n xxmaj sender : xxunk \n xxmaj organization : xxmaj ideology xxmaj busters , xxmaj inc . \n xxmaj lines : xxunk \n\n xxmaj this is the xxup index file for",16
2,"xxbos xxrep 10 - cut here xxrep 10 - part 03 / 03 \n xxup xxunk a \ - \ ( $ xxunk xxup xxunk < # xxup xxunk ( \n xxup xxunk u ) xxunk / xxup xxunk < xxunk # xxup xxunk xxup ad . "" / , ( , v1 # & w ; \n xxup xxunk xxunk : xxunk \ xxup xxunk % ! ] xxunk \n m / ' ' j - xxunk # \ xxunk # xxup xxunk ) \n xxup xxunk < xxunk . ; \ xxup xxunk = xxunk # xxunk \n xxup xxunk # xxup xxunk : al ' \n xxup xxunk : xxunk , xxunk ; \ ' xxunk \n xxup xxunk < hx - xxunk = xxunk / xxunk \n xxup mh = xxunk # xxup xxunk # xxunk - xxunk \n xxup xxunk / xxup xxunk : xxunk",2


In [14]:
#dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
fscore = F1Score(average='weighted')
ls_fucn = CrossEntropyLossFlat()
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, 
                                loss_func = ls_fucn,
                                metrics=[accuracy,fscore])
learn.fine_tune(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,nan,1.723637,0.472149,0.462963,01:10


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,nan,1.377396,0.564987,0.554876,02:05
1,nan,1.183050,0.622900,0.618430,02:05
